**Author:** Shahed Sabab  
**Tech Stack:** Neo4j, Langchain, Ollama  

---

## Content Summary

This notebook provides an implementation of a Hybrid Retrieval-Augmented Generation (RAG) model combining two approaches: Knowledge Graph-based retrieval and Naive RAG. The model leverages the strengths of graph databases like Neo4j for structured retrieval alongside unstructured document retrieval methods (naive RAG) to create a more efficient and versatile information retrieval system.

### Key Concepts:

1. **Knowledge Graph (KG) RAG**:
   - Utilizes graph-based data structures for storing and retrieving information.
   - Powered by Neo4j to enable the use of relationships and connections within the data to enhance query results.
   
2. **Naive RAG**:
   - A simple document-based RAG that relies on unstructured data.
   - This method allows the system to perform searches across a corpus without needing predefined relations.
   
3. **Langchain Integration**:
   - Facilitates the chaining of various modules in the pipeline for language model interaction.
   - Allows smooth interaction with Neo4j and the document retrieval system.
   
4. **Ollama Integration**:
   - Ollama models for local LLM support

5. **Gradio UI Integration**:
   - A user-friendly web interface is built using Gradio, allowing interaction with the Hybrid RAG model through a simple, accessible UI. 


---

<div style="text-align: center;">
    <img src="figure/GraphRAG.png" alt="Hybrid RAG Diagram" width="1200"/>
</div>

# Define packages 

In [124]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chains import GraphCypherQAChain
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Neo4jVector
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import  RunnablePassthrough
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import json 
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from neo4j import GraphDatabase
from langchain_experimental.llms.ollama_functions import OllamaFunctions
import gradio as gr
# Load environment variables from .env file
load_dotenv(override=True)

# Access the variables
neo4j_uri = os.getenv('NEO4J_URI')
neo4j_username = os.getenv('NEO4J_USERNAME')
neo4j_password = os.getenv('NEO4J_PASSWORD')

# Neo4j Setup

In [125]:

graph=Neo4jGraph()

# Local llm model


In [129]:
ollama_llm = OllamaFunctions(model="gemma2:9b", temperature=0, format="json")

In [131]:
llm=ollama_llm

# Text splitter

In [132]:

loader = TextLoader(file_path="input/dummy_text.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [133]:
documents

[Document(metadata={'source': 'input/dummy_text.txt'}, page_content='I met Geoffrey Hinton at his house on a pretty street in north London just four days before the bombshell \nannouncement that he is quitting Google. Hinton is a pioneer of deep learning who helped develop'),
 Document(metadata={'source': 'input/dummy_text.txt'}, page_content='some of the most important techniques at the heart of modern artificial intelligence, \nbut after a decade at Google, he is stepping down to focus on new concerns he now has about AI.'),
 Document(metadata={'source': 'input/dummy_text.txt'}, page_content='Stunned by the capabilities of new large language models, \nHinton wants to raise public awareness of the serious risks that he now believes may accompany the technology he ushered in.'),
 Document(metadata={'source': 'input/dummy_text.txt'}, page_content='At the start of our conversation, I took a seat at the kitchen table, and Hinton started pacing. \nPlagued for years by chronic back pain, Hi

# Graph Transformer (convert documents-> graph documents)

In [134]:
# Initialize the LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Convert the document to a graph
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [135]:
graph_documents[1]

GraphDocument(nodes=[Node(id='Some Of The Most Important Techniques At The Heart Of Modern Artificial Intelligence', type='Concept', properties={}), Node(id='He', type='Person', properties={}), Node(id='Google', type='Organization', properties={})], relationships=[Relationship(source=Node(id='He', type='Person', properties={}), target=Node(id='Some Of The Most Important Techniques At The Heart Of Modern Artificial Intelligence', type='Concept', properties={}), type='RELATED_TO', properties={}), Relationship(source=Node(id='He', type='Person', properties={}), target=Node(id='Google', type='Organization', properties={}), type='WORKED_AT', properties={})], source=Document(metadata={'source': 'input/dummy_text.txt'}, page_content='some of the most important techniques at the heart of modern artificial intelligence, \nbut after a decade at Google, he is stepping down to focus on new concerns he now has about AI.'))

In [136]:
graph_documents[1].nodes

[Node(id='Some Of The Most Important Techniques At The Heart Of Modern Artificial Intelligence', type='Concept', properties={}),
 Node(id='He', type='Person', properties={}),
 Node(id='Google', type='Organization', properties={})]

In [137]:
graph_documents[1].relationships

[Relationship(source=Node(id='He', type='Person', properties={}), target=Node(id='Some Of The Most Important Techniques At The Heart Of Modern Artificial Intelligence', type='Concept', properties={}), type='RELATED_TO', properties={}),
 Relationship(source=Node(id='He', type='Person', properties={}), target=Node(id='Google', type='Organization', properties={}), type='WORKED_AT', properties={})]

# Push graph to Neo4j

In [138]:
# Use the add_graph_documents method to push the data
graph.add_graph_documents(
    graph_documents=graph_documents,  # Your graph_document goes here
    include_source=True,  # Set to True if you want to include the source document
    baseEntityLabel=True  # Set to True to add a base label to all entities
)

print("Graph data has been successfully pushed to Neo4j.")

Graph data has been successfully pushed to Neo4j.


# Vector retriever

In [139]:
ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [140]:
db = Neo4jVector.from_documents(
    documents, 
    ollama_embeddings, 
    url=neo4j_uri,
    username=neo4j_username, 
    password=neo4j_password
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding) SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS "


In [141]:
query = "who is hinton"
docs_with_score = db.similarity_search_with_score(query, k=1)
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.8407111167907715
Widely regarded as the “godfather of AI,” Hinton shared the Noble prize with John J. Hopfield 
for foundational discoveries and inventions that enable machine learning with artificial neural networks.
--------------------------------------------------------------------------------


In [142]:
vector_index = Neo4jVector.from_existing_graph(
    ollama_embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


In [157]:
def vector_retriever(question, top_k=1):
    vector_ret = vector_index.as_retriever(k=top_k)
    return [el.page_content for el in vector_ret.invoke(question)][:top_k]

In [159]:
driver = GraphDatabase.driver(
    uri = neo4j_uri,
    auth = (neo4j_username,
            neo4j_password))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

# Graph retriever

In [160]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [161]:
entity_chain.invoke("Who is hinton?")

Entities(names=['hinton'])

In [162]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    # detect entity through entity chain and pass it to graph query
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [163]:

print(graph_retriever("Who is hinton"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 129} for query: "\n            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Hinton - STUNNED_BY -> Large Language Models
Hinton - AT -> Kitchen Table
Hinton - BELIEVES -> Risks
Geoffrey Hinton - WORKS_FOR -> Google
Geoffrey Hinton - SHARED_AWARD -> Yann Lecun
Geoffrey Hinton - SHARED_AWARD -> Yoshua Bengio
Geoffrey Hinton - RECEIVED -> Turing Award
Geoffrey Hinton - LIVES_IN -> North London
Geoffrey Hinton - SHARED_NOBLE_PRIZE -> John J. Hopfield


# Hybrid Retriever

In [164]:
def hybrid_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = vector_retriever(question)
    final_data = f"""
    GRAPH DATA:
    {graph_data}
    VECTOR DATA:
    {"#Document ". join(vector_data)}
    """
    return {
        "final_data": final_data,
        "graph_data": graph_data,
        "vector_data": vector_data
    }

In [165]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
Only generate your response from the context. Do not make anything up.
Add as much information as needed to generate a coherent and informative response 
based on the context. 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": lambda x: hybrid_retriever(x)["final_data"],
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [166]:
def invoke_chain(query):
    retriever_output = hybrid_retriever(query)
    response = chain.invoke(query)
    return {
        "response": response,
        "graph_data": retriever_output["graph_data"],
        "vector_data": retriever_output["vector_data"]
    }

In [167]:
def gradio_interface(query):
    result = invoke_chain(query)
    return (
        result["response"],
        str(result["graph_data"]),
        "\n".join(result["vector_data"])
    )

with gr.Blocks() as demo:
    gr.Markdown("# GraphRAG Query Interface")
    
    query_input = gr.Textbox(lines=2, placeholder="Enter your query here...")
    submit_btn = gr.Button("Submit")
    
    with gr.Column():
        response_output = gr.Textbox(label="Model Response")
        
        with gr.Accordion("Graph Data", open=True):
            graph_data_output = gr.Textbox(label="Graph Data")
        
        with gr.Accordion("Vector Data", open=True):
            vector_data_output = gr.Textbox(label="Vector Data")
    
    submit_btn.click(
        fn=gradio_interface,
        inputs=query_input,
        outputs=[response_output, graph_data_output, vector_data_output]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 129} for query: "\n            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARN